In [1]:
from ansys.mapdl.core import launch_mapdl
import numpy as np
import pyvista as pv
import numpy as np
import matplotlib.pyplot as plt
np.set_printoptions(precision=4)
import pandas as pd
import pickle
import os
import sys

In [2]:
mapdl = launch_mapdl()
mapdl.clear()
sample_dim= np.array([50e-3, 50e-3,4e-3])
sample_mat=np.array ([148, 2330,710])
sample_dim= sample_dim.reshape((1,3))
sample_mat= sample_mat.reshape((1,3))

sample_x = sample_dim[0,0]
sample_y = sample_dim[0, 1]
sample_z = sample_dim[0,2]

kxx= sample_mat[0,0]
dens = sample_mat[0,1]
c= sample_mat[0,2]

Q = 1000
freq = 0.182
#freq = 1
T_wave = 1 / freq  
omega = 2 * np.pi * freq  
t_end = 5 * T_wave 
room_temp = 25
num_steps_per_wave = 10

 
deltim = T_wave / t_end 

num_steps = int(t_end / deltim)
time_values = np.linspace(0, t_end, num_steps, endpoint=False)
cos_values = np.cos(omega * time_values)
heat_flux = Q * (1 - cos_values)

# Store time and heat_flux in the load_table
load_table = np.column_stack((time_values, heat_flux))

delta_x = np.sqrt((deltim * 100 * kxx) / (dens * c))

mapdl.clear()
mapdl.prep7()

mapdl.et(1,70)
mapdl.mp(lab="KXX", mat=1 , c0=kxx)
mapdl.mp(lab='DENS', mat=1, c0=dens)
mapdl.mp(lab='C', mat=1, c0=c)

PyMAPDL is taking longer than expected to connect to an MAPDL session.
Checking if there are any available licenses...


MATERIAL          1     C    =   477.0000

In [3]:
A = [
    {"x_strat": 30e-3, "x_end": 35e-3,  "y_strat": 30e-3, "y_end": 35e-3,  "z_strat": 4e-3, "z_end": 3e-3 },
    {"x_strat": 70e-3, "x_end": 80e-3,  "y_strat": 70e-3, "y_end": 80e-3,  "z_strat": 4e-3, "z_end": 2e-3}]


numeric_values_A = [[item["x_strat"], item["x_end"], item["y_strat"], item["y_end"], item["z_strat"],  item["z_end"]] for item in A]
    
defects_loc = np.array(numeric_values_A)

In [4]:

#x_start= 30e-3, 
#x_end= 40e-3, 
#y_start= 30e-3, 
#y_end=40e-3, 
#z_start= -2e-3, 
#z_end= -10e-3

for data_point in defects_loc:
    x_start = data_point[0]     
    x_end = data_point[1]  

    y_start = data_point[2]     
    y_end = data_point[3] 

    z_start = data_point[4]     
    z_end = data_point[5] 

    rect_vol = mapdl.block(0, sample_x, 0, sample_y, 0, sample_z)
    Rect_complete = mapdl.block(x1=x_start, x2=x_end, y1=y_start, y2=y_end, z1=0, z2=sample_z)
    Rect_partial = mapdl.block(x1=x_start, x2=x_end, y1=y_start, y2=y_end, z1=z_start, z2=z_end)
    mapdl.vsel('none')
    mapdl.vsel('A', vmin=rect_vol)
    mapdl.vsel('A', vmin=Rect_complete)
    mapdl.vsbv(nv1=rect_vol, nv2=Rect_complete)
    mapdl.allsel()
    mapdl.vglue("all")
    break
mapdl.vplot("all")

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [5]:
mapdl.vsel("all")

array([1, 3])

In [6]:
mapdl.allsel()
mapdl.esize(size=delta_x)
mapdl.vsweep(vnum=rect_vol)
mapdl.mshape(1, "3D")
mapdl.mshkey(0)
mapdl.vmesh("all")


GENERATE NODES AND ELEMENTS   IN  ALL  SELECTED VOLUMES  

 *** NOTE ***                            CP =       0.141   TIME= 18:43:05
 4 Pyramid transition elements automatically defined in volume 3 at      
 boundary between brick (or quad) and tetrahedral elements.              

 NUMBER OF VOLUMES MESHED   =         1
 MAXIMUM NODE NUMBER        =       170
 MAXIMUM ELEMENT NUMBER     =       106

In [7]:
mapdl.vplot()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)